In [15]:
import requests
import json

In [16]:
from gamma_viewer import GammaViewer
from IPython.display import display

In [17]:
def one_hop_message(curiea,typea,typeb,edgetype,reverse=False):
    query_graph = {
    "nodes": [
        {
            "id": "a",
            "type": typea,
            "curie": curiea
        },
        {
            "id": "b",
            "type": typeb
        }
    ],
    "edges": [
        {
            "id": "ab",
            "source_id": "a",
            "target_id": "b"
        }
    ]
    }
    if edgetype is not None:
        query_graph['edges'][0]['type'] = edgetype
        if reverse:
            query_graph['edges'][0]['source_id'] = 'b'
            query_graph['edges'][0]['target_id'] = 'a'
    message = {"message": {"query_graph": query_graph,
                          'knowledge_graph':{"nodes": [], "edges": [],}, 
                           'results':[]}}
    return message

In [31]:
def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message['message'])
    print(response.status_code)
    return response.json()

def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        return {}
    return response.json()

def strider(message):
    url = 'http://robokop.renci.org:5781/query'
    strider_answer = post(strider,url,message)
    numanswers = len(strider_answer['results'])
    if (numanswers == 0) or ((numanswers == 1) and (len(strider_answer['results'][0]['node_bindings'])==0)):
        print('no answers')
        return {}
    #Strider for some reason doesn't return the query graph
    strider_answer['query_graph'] = message['message']['query_graph']
    return strider_answer
    
def striderandfriends(message):
    strider_answer = strider(message)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',{'message': strider_answer})
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',{'message': omni_answer})
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',{'message': weighted_answer})
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/graph',{'message':scored_answer})
    return coalesced_answer

def aragorn(message, coalesce_type):
    answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce':coalesce_type})
    return answer

Send this message directly to aragorn with graph coalesence

In [40]:
message = one_hop_message('MONDO:0005090','disease','chemical_substance','treats',reverse=True)
results=aragorn(message,'graph')

Check the query graph to see if any new nodes have been added.  Nope.

In [41]:
results['query_graph']

{'nodes': [{'id': 'a', 'curie': 'MONDO:0005090', 'type': 'disease'},
  {'id': 'b', 'type': 'chemical_substance'}],
 'edges': [{'id': 'ab', 'source_id': 'b', 'target_id': 'a', 'type': 'treats'}]}

In [ ]:
#Not relevant for the current question
#view = GammaViewer(props={"data":results})
#display(view)

Now do the same thing calling the services individually

In [32]:
message = one_hop_message('MONDO:0005090','disease','chemical_substance','treats',reverse=True)
results=striderandfriends(message)

And check the query graph.  Now we are getting an extra node in the qg, indicating that AC did something.

In [34]:
results['query_graph']

{'nodes': [{'id': 'a', 'curie': 'MONDO:0005090', 'type': 'disease'},
  {'id': 'b', 'type': 'chemical_substance', 'set': True},
  {'id': 'extra_qn_0',
   'type': ['named_thing',
    'biological_entity',
    'disease',
    'disease_or_phenotypic_feature']}],
 'edges': [{'id': 'ab', 'source_id': 'b', 'target_id': 'a', 'type': 'treats'},
  {'id': 'extra_qe_0', 'source_id': 'b', 'target_id': 'extra_qn_0'}]}

Fin.

In [6]:
#Target for aspirin
message= one_hop_message('CHEBI:15365','chemical_substance','gene','related_to')
results = striderandfriends(message)

In [26]:
print(json.dumps(results,indent=4))

{}


In [7]:
#genes that cause cystic fibrosis
message= one_hop_message('MONDO:0009061','disease','gene','causes',reverse=True)
results = striderandfriends(message)

In [8]:
#phenotypes of Charcot Marie Tooth
message = one_hop_message('MONDO:0015626','disease','phenotypic_feature','has_phenotype')
results = striderandfriends(message)

In [9]:
#What cells are in the brain?
message = one_hop_message('UBERON:0000955','anatomical_entity','cell','part_of',reverse=True)
results = striderandfriends(message)

In [10]:
#which genes enable glycolipid binding?
message = one_hop_message('GO:0051861','molecular_activity','gene','enabled_by')
results = striderandfriends(message)

In [17]:
#what part of the body does vasculitis affect?
message = one_hop_message('MONDO:0018882','disease','anatomical_entity','biolink:affects')
results = automat('uberongraph',message)
#results = striderandfriends(message)
print(message)
print(results)

500
{'message': {'query_graph': {'nodes': [{'id': 'a', 'type': 'disease', 'curie': 'MONDO:0018882'}, {'id': 'b', 'type': 'anatomical_entity'}], 'edges': [{'id': 'ab', 'source_id': 'a', 'target_id': 'b', 'type': 'biolink:affects'}]}, 'knowledge_graph': {'nodes': [], 'edges': []}, 'results': []}}
{'error': ''}


In [12]:
#what are the types of diabetes?
message = one_hop_message('MONDO:0005015','disease','disease','subclass_of',reverse=True)
results = striderandfriends(message)

no answers
{'query_graph': {'nodes': [{'id': 'a', 'curie': 'MONDO:0005015', 'type': 'disease'}, {'id': 'b', 'curie': None, 'type': 'disease'}], 'edges': [{'id': 'ab', 'source_id': 'b', 'target_id': 'a', 'type': 'subclass_of'}]}, 'knowledge_graph': {'nodes': [], 'edges': []}, 'results': []}
